In [1]:
import pandas as pd
import numpy as np

In [2]:
countries_codes = pd.DataFrame.from_csv(r"c:\Dev\05. Jupyter\2. FDA countries clean\csv\countries_codes_curated_1Nov.csv")
countries_codes.head()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,iso2,full_country_name,comment,wiki_link,population,area
0,no_code_**none**,**None**,NaN,NaN,1,1
1,af,Afghanistan,NaN,https://en.wikipedia.org/wiki/Afghanistan,1,1
2,al,Albania,NaN,https://en.wikipedia.org/wiki/Albania,1,1
3,dz,Algeria,NaN,https://en.wikipedia.org/wiki/Algeria,1,1
4,as,American Samoa,NaN,NaN,1,1


In [4]:
from bs4 import BeautifulSoup
import requests
import logging, hashlib, time, os, os.path, codecs

logging.basicConfig(filename=r'c:\Dev\04. Python\04. Sqlite converter for clinicaltrials.gov\01-Nov-2018_bis.log',level=logging.DEBUG)


def getPage(page_url, BASE_PATH, time_to_sleep_before_network = 10):
    logging.debug("Page URL:{}".format(page_url))
    print(page_url)
    
    file_to_save = BASE_PATH+  \
                        page_url.replace("/", "_").replace("?", "_").replace(":", "_")[:50]+ \
                        "_"+hashlib.sha224(page_url.encode('utf-8')).hexdigest()+ \
                        ".html"
    logging.debug("File name:{}".format(file_to_save.encode("utf-8")))
    if os.path.exists(file_to_save):
        logging.debug("File exists returning from disk")
        f=codecs.open(file_to_save, "r", encoding="utf-8")
        r = f.read()
        f.close()
        return r

    logging.debug("File does not exist: downloading")
    print("File does not exist: downloading")
    time.sleep(time_to_sleep_before_network)
    print("Sleep over")
    r = requests.get(page_url)
    #https://stackoverflow.com/questions/934160/write-to-utf-8-file-in-python
    f = codecs.open(file_to_save, "w", "utf-8")
    f.write(r.text)
    f.close()
    return r.text



In [31]:
import re


def parsePageForArea(pageLink):
    page_content = getPage(pageLink, "c:\\Dev\\05. Jupyter\\2. FDA countries clean\\", 10)
#     print(page_content)
    soup = BeautifulSoup(page_content)
    rows = soup.select(".infobox")
    
    for row in rows[0].find_all("tr"):
        found = False
#         print(row)
        area_text = "999" #This is a very abnormal return
        link = row.select("th a")
        if len(link)>0:
            if link[0].text.strip().lower() == "area":
                print(link)
                print(link[0].parent.parent.next_sibling)
                area_text = link[0].parent.parent.next_sibling.text
                found = True
        else:#for come countries: Dominican Republic, Libyan Arab Jamahiriya, Mali and United Arab Emirates
                #different setup of tags
            area_tag = row.select("th")
            if len(area_tag)>0:
                if area_tag[0].text.strip().lower() == "area":
                    print("Area tag:{}".format(area_tag))
                    print("Area tag:{}".format(area_tag[0].parent))
                    print("Area tag:{}".format(area_tag[0].parent.parent))
                    print("Area tag:{}".format(area_tag[0].parent.next_sibling))
                    area_text =  area_tag[0].parent.next_sibling.select("td")[0].text
                    print(area_text)
                    found = True
                
        if found:
            print("Area_text:{}".format(area_text))
            #Matching against area_text
            matchObj = re.search( r'([\d,]{1,})', area_text)#, re.M|re.I)

            if matchObj:
                print("matchObj.group() :{} ".format(matchObj.group()))
                print("matchObj.group(0) :{} ".format(matchObj.group(0))) 
                print("matchObj.group(1) :{} ".format(matchObj.group(1)))
                return int(matchObj.group().replace(",", ""))
    #                print("matchObj.group() :{} ".format(matchObj.group(2)))     
            else:
                print("No match!!")
                return -1
        
    return -2 #This is abnormal return, but it should prevent None in resulting list
    #END for row in rows[0].find_all("tr"):
                
#END def parsePageForArea(pageLink):        

In [32]:
parsePageForArea("https://en.wikipedia.org/wiki/Dominican_Republic")

https://en.wikipedia.org/wiki/Dominican_Republic
Area tag:[<th colspan="2" style="text-align:center;text-align:left;">Area</th>]
Area tag:<tr class="mergedtoprow"><th colspan="2" style="text-align:center;text-align:left;">Area</th></tr>
Area tag:<tbody><tr><th class="adr" colspan="2" style="text-align:center;font-size:125%;font-weight:bold;padding:0.25em 0.33em 0.33em;line-height:1.2em;font-size:1.25em;"><span class="fn org country-name">Dominican Republic</span><br/><div style="padding-top:0.25em; font-weight:normal;"><i lang="es" title="Spanish language text">República Dominicana</i>  <span class="languageicon" style="font-size:85%;font-weight:normal;">(<a href="/wiki/Spanish_language" title="Spanish language">Spanish</a>)</span></div></th></tr><tr><td colspan="2" style="text-align:center;text-align:center;padding:0.5em 0;">
<div style="display:table; width:100%;">
<div style="display:table-cell; width:58%; vertical-align:middle; padding-left:5px;">
<div style="padding-bottom:3px;"><

78315

In [33]:
result_areas_array = []
# for country in countries_codes.loc[countries_codes["wiki_link"].isnull()==False].values:
for country in countries_codes.values:
    print(country)
    if type(country[3]) is float:
        result_areas_array.append(-1)
    else:
        link = country[3]
        print("Iterating link:{}".format(link))
        areaResult = parsePageForArea(link)
        result_areas_array.append(areaResult)
        print(areaResult)

['no_code_**none**' '**None**' nan nan 1 1]
['af' 'Afghanistan' nan 'https://en.wikipedia.org/wiki/Afghanistan' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Afghanistan
https://en.wikipedia.org/wiki/Afghanistan
[<a href="/wiki/Geography_of_Afghanistan" title="Geography of Afghanistan">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
652,864<sup class="reference" id="cite_ref-Area_and_administrative_Population_6-0"><a href="#cite_note-Area_and_administrative_Population-6">[5]</a></sup> km<sup>2</sup> (252,072 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">40th</a>)</td></tr>
Area_text:• Total
652,864[5] km2 (252,072 sq mi) (40th)
matchObj.group() :652,864 
matchObj.group(0) :652,864 
matchObj.group(1) :652,864 
652864
['al' 'Albania' nan 'https://en.wikipedia.org/wiki/Albania' 1 1]
Iterating link:https://en.wikipedia.org

[<a href="/wiki/Geography_of_Belarus" title="Geography of Belarus">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
207,595 km<sup>2</sup> (80,153 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">84th</a>)</td></tr>
Area_text:• Total
207,595 km2 (80,153 sq mi) (84th)
matchObj.group() :207,595 
matchObj.group(0) :207,595 
matchObj.group(1) :207,595 
207595
['be' 'Belgium' nan 'https://en.wikipedia.org/wiki/Belgium' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Belgium
https://en.wikipedia.org/wiki/Belgium
[<a href="/wiki/Geography_of_Belgium" title="Geography of Belgium">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
30,528 km<sup>2</sup> (11,787 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of cou

[<a href="/wiki/Geography_of_Burundi" title="Geography of Burundi">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
27,834 km<sup>2</sup> (10,747 sq mi)<sup class="reference" id="cite_ref-ISTEEBU_7-0"><a href="#cite_note-ISTEEBU-7">[7]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">142nd</a>)</td></tr>
Area_text:• Total
27,834 km2 (10,747 sq mi)[7] (142nd)
matchObj.group() :27,834 
matchObj.group(0) :27,834 
matchObj.group(1) :27,834 
27834
['kh' 'Cambodia' nan 'https://en.wikipedia.org/wiki/Cambodia' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Cambodia
https://en.wikipedia.org/wiki/Cambodia
[<a href="/wiki/Geography_of_Cambodia" title="Geography of Cambodia">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
181,035 km<sup>2<

[<a href="/wiki/Geography_of_Denmark" title="Geography of Denmark">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Metropolitan</div></th><td>
42,933 km<sup>2</sup> (16,577 sq mi)<sup class="reference" id="cite_ref-area_7-0"><a href="#cite_note-area-7">[3]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">130th</a>)</td></tr>
Area_text:• Metropolitan
42,933 km2 (16,577 sq mi)[3] (130th)
matchObj.group() :42,933 
matchObj.group(0) :42,933 
matchObj.group(1) :42,933 
42933
['dj' 'Djibouti' nan 'https://en.wikipedia.org/wiki/Djibouti' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Djibouti
https://en.wikipedia.org/wiki/Djibouti
[<a href="/wiki/Geography_of_Djibouti" title="Geography of Djibouti">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
23,200<su

[<a href="/wiki/Geography_of_Egypt" title="Geography of Egypt">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
1,010,408<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[4]</a></sup> km<sup>2</sup> (390,121 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">29th</a>)</td></tr>
Area_text:• Total
1,010,408[4] km2 (390,121 sq mi) (29th)
matchObj.group() :1,010,408 
matchObj.group(0) :1,010,408 
matchObj.group(1) :1,010,408 
1010408
['sv' 'El Salvador' nan 'https://en.wikipedia.org/wiki/El_Salvador' 1 1]
Iterating link:https://en.wikipedia.org/wiki/El_Salvador
https://en.wikipedia.org/wiki/El_Salvador
[<a href="/wiki/Geography_of_El_Salvador" title="Geography of El Salvador">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
21,0

...

[<a href="/wiki/Geography_of_New_Zealand" title="Geography of New Zealand">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
268,021 km<sup>2</sup> (103,483 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">75th</a>)</td></tr>
Area_text:• Total
268,021 km2 (103,483 sq mi) (75th)
matchObj.group() :268,021 
matchObj.group(0) :268,021 
matchObj.group(1) :268,021 
268021
['ni' 'Nicaragua' nan 'https://en.wikipedia.org/wiki/Nicaragua' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Nicaragua
https://en.wikipedia.org/wiki/Nicaragua
[<a href="/wiki/Geography_of_Nicaragua" title="Geography of Nicaragua">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
130,375 km<sup>2</sup> (50,338 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_a

https://en.wikipedia.org/wiki/Philippines
[<a href="/wiki/Geography_of_the_Philippines" title="Geography of the Philippines">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
300,000<sup class="reference" id="cite_ref-psa.gov.ph_4-0"><a href="#cite_note-psa.gov.ph-4">[4]</a></sup><sup class="reference" id="cite_ref-worldbank.org_5-0"><a href="#cite_note-worldbank.org-5">[5]</a></sup> km<sup>2</sup> (120,000 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">72nd</a>)</td></tr>
Area_text:• Total
300,000[4][5] km2 (120,000 sq mi) (72nd)
matchObj.group() :300,000 
matchObj.group(0) :300,000 
matchObj.group(1) :300,000 
300000
['pl' 'Poland' nan 'https://en.wikipedia.org/wiki/Poland' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Poland
https://en.wikipedia.org/wiki/Poland
[<a href="/wiki/Geography_of_Poland" title="Geography of P

[<a href="/wiki/Geography_of_Sierra_Leone" title="Geography of Sierra Leone">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
71,740 km<sup>2</sup> (27,700 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">117th</a>)</td></tr>
Area_text:• Total
71,740 km2 (27,700 sq mi) (117th)
matchObj.group() :71,740 
matchObj.group(0) :71,740 
matchObj.group(1) :71,740 
71740
['sg' 'Singapore' nan 'https://en.wikipedia.org/wiki/Singapore' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Singapore
https://en.wikipedia.org/wiki/Singapore
[<a href="/wiki/Geography_of_Singapore" title="Geography of Singapore">Area </a>]
<tr class="mergedbottomrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
722.5 km<sup>2</sup> (279.0 sq mi)<sup class="reference" id="cite_ref-total_area_of_Singapo

[<a href="/wiki/Geography_of_Tajikistan" title="Geography of Tajikistan">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
143,100 km<sup>2</sup> (55,300 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">94th</a>)</td></tr>
Area_text:• Total
143,100 km2 (55,300 sq mi) (94th)
matchObj.group() :143,100 
matchObj.group(0) :143,100 
matchObj.group(1) :143,100 
143100
['tz' 'Tanzania' nan 'https://en.wikipedia.org/wiki/Tanzania' 1 1]
Iterating link:https://en.wikipedia.org/wiki/Tanzania
https://en.wikipedia.org/wiki/Tanzania
[<a href="/wiki/Geography_of_Tanzania" title="Geography of Tanzania">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
947,303 km<sup>2</sup> (365,756 sq mi) (<a href="/wiki/List_of_countries_and_dependencies_by_area" titl

[<a href="/wiki/Geography_of_the_United_Kingdom" title="Geography of the United Kingdom">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;font-weight:normal;">• Total</div></th><td>
242,495 km<sup>2</sup> (93,628 sq mi)<sup class="reference" id="cite_ref-12"><a href="#cite_note-12">[8]</a></sup> (<a href="/wiki/List_of_countries_and_dependencies_by_area" title="List of countries and dependencies by area">78th</a>)</td></tr>
Area_text:• Total
242,495 km2 (93,628 sq mi)[8] (78th)
matchObj.group() :242,495 
matchObj.group(0) :242,495 
matchObj.group(1) :242,495 
242495
['us' 'United States of America' nan
 'https://en.wikipedia.org/wiki/United_States' 1 1]
Iterating link:https://en.wikipedia.org/wiki/United_States
https://en.wikipedia.org/wiki/United_States
[<a href="/wiki/Geography_of_the_United_States" title="Geography of the United States">Area </a>]
<tr class="mergedrow"><th scope="row"><div style="text-indent:-0.9em;margin-left:1.2em;f

In [34]:
# i=0
# while i<len(result_areas_array):
#     if result_areas_array[i] is None:
#         result_areas_array[i] = -1
#     i+=1

print(result_areas_array)

[-1, 652864, 28748, 2381741, -1, 467, 1246700, -1, 29743, -1, 7692024, 83879, 86600, -1, 765, 147570, 439, 207595, 30528, 22966, 114763, -1, 38394, 1098581, 51129, 581730, 8515767, 5765, 110993, 274200, 27834, 181035, 475442, -1, -1, 622984, 1284000, 756096, -1, 1141748, 1862, -1, 51100, 56594, 109884, 9251, -1, -1, 42933, 23200, 78315, 283561, 1010408, 21041, 28050, 45227, 1104300, -1, -1, 18274, 338424, -1, -1, 640679, -1, -1, 267667, -1, 69700, 357386, 239567, -1, 131957, -1, 348, -1, -1, 108889, -1, 36125, 214970, 27750, 0, 112492, -1, 93030, -1, 3287263, 1904569, 1648195, 437072, -1, 20770, 301340, 10991, 377973, -1, 89342, 2724900, 580367, 120540, 100363, -1, 17818, 199951, 237955, 64589, 10452, 30355, 111369, 1759541, 160, 65300, 2586, 25713, 587041, 118484, 330803, 298, 1240192, 316, -1, 1030000, 2040, -1, 1972550, 33846, 2, 1566000, 13812, -1, 710850, 801590, -1, 825615, 147181, 42508, -1, -1, 268021, 130375, 1267000, 923768, -1, 385203, 309500, 881913, 459, 6020, -1, 75417, 4

In [35]:
countries_codes["area"]=np.array(result_areas_array).astype(int)
countries_codes.head()
# countries_codes.loc[countries_codes.full_country_name == "Qatar"].head()
# countries_codes.loc[countries_codes.area.isnull()].head()

,iso2,full_country_name,comment,wiki_link,population,area
0,no_code_**none**,**None**,NaN,NaN,1,-1
1,af,Afghanistan,NaN,https://en.wikipedia.org/wiki/Afghanistan,1,652864
2,al,Albania,NaN,https://en.wikipedia.org/wiki/Albania,1,28748
3,dz,Algeria,NaN,https://en.wikipedia.org/wiki/Algeria,1,2381741
4,as,American Samoa,NaN,NaN,1,-1


In [36]:
countries_codes.to_csv(r"c:\Dev\05. Jupyter\2. FDA countries clean\csv\countries_codes_area_1Nov.csv")